<a href="https://colab.research.google.com/github/christianiturnip/Midterm-Christiani-Turnip-2206130694/blob/main/Mid_Term_ADL_Christiani_Turnip_2206130694.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the required libraries
!pip install torch torchvision torchaudio
!pip install opencv-python
!pip install -U git+https://github.com/ultralytics/yolov5.git

# Set GPU as the runtime
import torch

# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import torch

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)  # Load a pre-trained YOLOv5 model

# Define the video path
video_path = '/content/drive/MyDrive/vidioadl.mp4'  # Replace with your video path

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import torch

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)  # Load a pre-trained YOLOv5 model

# Define the video path
video_path = '/content/drive/MyDrive/vidioadl.mp4'  # Replace with your video path

# Open the video file
cap = cv2.VideoCapture(video_path)

# Process each frame
frame_count = 0
max_frames = 100  # Set a limit for frames to process

while cap.isOpened() and frame_count < max_frames:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference on the frame
    results = model(frame)

    # Extract bounding boxes and class labels
    bbox_xyxy = results.xyxy[0]  # Bounding boxes
    for *bbox, conf, cls in bbox_xyxy:  # bbox format: [x1, y1, x2, y2, confidence, class]
        x1, y1, x2, y2 = map(int, bbox)  # Convert to integers
        label = model.names[int(cls)]  # Get class label

        # Draw bounding box on the frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Display the frame using cv2_imshow
    cv2_imshow(frame)

    frame_count += 1

# Release the video capture object
cap.release()

In [ ]:
# Store detection results
detections = []

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    bbox_xyxy = results.xyxy[0]

    # Preprocess the results
    for *bbox, conf, cls in bbox_xyxy:
        x1, y1, x2, y2 = map(int, bbox)
        label = model.names[int(cls)]
        detection = {
            'frame': frame_number,  # frame number
            'bounding_box': [x1, y1, x2, y2],
            'confidence': conf.item(),
            'class_label': label
        }
        detections.append(detection)

    frame_number += 1

# Save the detections for analysis
import pandas as pd
detections_df = pd.DataFrame(detections)
detections_df.to_csv('/content/drive/MyDrive/detections.csv', index=False)

In [ ]:
# Simple tracking logic (not using a dedicated tracking library)
tracked_objects = {}

# Iterate through the detections
for detection in detections:
    frame_number = detection['frame']
    bbox = detection['bounding_box']
    label = detection['class_label']

    # Maintain object identity
    if label not in tracked_objects:
        tracked_objects[label] = []

    tracked_objects[label].append({
        'frame': frame_number,
        'bounding_box': bbox
    })

# Example of accessing tracking data
for obj, tracks in tracked_objects.items():
    print(f'Tracking {obj}: {len(tracks)} frames')

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, force_reload=True)
model.eval()  # Set to evaluation mode

In [ ]:
frame_skip = 2  # Process every 2nd frame
frame_number = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_number % frame_skip == 0:
        results = model(frame)
        # Processing results...
    frame_number += 1